<a href="https://colab.research.google.com/github/SWATHIRAVIRS/githubr/blob/main/textgenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical


In [2]:
with open("/content/next_word_predictor.txt","r",encoding="utf8") as file:
  text=file.read().lower()
print("Dataset Loaded")
print(text[:200])


Dataset Loaded
the sun was shining brightly in the clear blue sky, and a gentle breeze rustled the leaves of the tall trees. people were out enjoying the beautiful weather, some sitting in the park, others taking a 


In [3]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])

total_word=len(tokenizer.word_index)+1
print("Total Word:",total_word)

Total Word: 4994


In [4]:
#sequence
input_sequences=[]
for line in text.split('\n'):
  token_list=tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
    n_gram=token_list[:i+1]
    input_sequences.append(n_gram)
print(input_sequences[:10])

[[1, 155], [1, 155, 21], [1, 155, 21, 2368], [1, 155, 21, 2368, 1549], [1, 155, 21, 2368, 1549, 8], [1, 155, 21, 2368, 1549, 8, 1], [1, 155, 21, 2368, 1549, 8, 1, 422], [1, 155, 21, 2368, 1549, 8, 1, 422, 692], [1, 155, 21, 2368, 1549, 8, 1, 422, 692, 215], [1, 155, 21, 2368, 1549, 8, 1, 422, 692, 215, 2]]


In [5]:
#padding
max_len=max([len(x) for x in input_sequences])
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_len,padding='pre'))

In [8]:
X=input_sequences[:,:-1]
y=input_sequences[:,-1]

y=to_categorical(y,num_classes=total_word)

print("X shape:",X.shape)
print("y shape:",y.shape)

X shape: (26383, 324)
y shape: (26383, 4994)


In [13]:
model=Sequential([
   Embedding(total_word,64,input_length=max_len-1),
   LSTM(100),
   Dense(total_word,activation='softmax')
]
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [14]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.fit(X,y,epochs=10,batch_size=512,verbose=30)

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


In [19]:
def predict_next_word(seed_text):
  token_list=tokenizer.texts_to_sequences([seed_text])[0]
  token_list=pad_sequences([token_list],maxlen=max_len-1,padding='pre')
  predicted=model.predict(token_list,verbose=0)
  predicted_index=np.argmax(predicted)

  for word,index in tokenizer.word_index.items():
    if index==predicted_index:
      return word

In [ ]:
print("\nMini GPT Ready 🥰 Type something\n")

while True:
    sentence = input("You: ")

    next_word = predict_next_word(sentence)

    print("Bot:", sentence + " " + next_word)


Mini GPT Ready 🥰 Type something

You: are
Bot: are the
You: most
Bot: most the
You: beautiful
Bot: beautiful the
You: person
Bot: person well
You: hi
Bot: hi the
You: the
Bot: the land
You: is
Bot: is the
You: sea
Bot: sea the
You: hello
Bot: hello oh
You: na
Bot: na the
You: wow
Bot: wow oh
You: i
Bot: i i
You: love
Bot: love the
You: you
Bot: you oh
